# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read in csv file and covert it to a dataframe
file = "../WeatherPy/city_data.csv"
df = pd.read_csv(file)
df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max temp,Wind Speed
0,Yellowknife,75,CA,1592241890,62,62.46,-114.35,57.20,24.16
1,Aswān,0,EG,1592242255,9,24.09,32.91,100.40,8.05
2,Upernavik,100,GL,1592242029,88,72.79,-56.15,34.54,2.15
3,Varca,75,IN,1592242255,83,15.22,73.92,84.20,12.75
4,Antonina,90,BR,1592242255,77,-25.43,-48.71,60.80,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)
#Set locations equal to lat and long
locations = df[["Lat","Long"]]
#Convert humidity to a float 
humidity = df["Humidity"].astype(float)

In [4]:
#Create figure
fig = gmaps.figure()
#Create heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,dissipating=False, max_intensity=10,
                                 point_radius=1)
#Add heat layer to map
fig.add_layer(heat_layer)
#Show figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#Narrow down dataframe to fit good weather conditions
#These include cities with max temp lower than 80 degrees, wind speed less than 10 mph and zero cloudiness
good_weath = df.loc[(df["Max temp"]<80) & (df["Wind Speed"]<10) & (df["Cloudiness"]==0),:]
#Drop any rows with null values
good_weath = good_weath.dropna()
good_weath.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max temp,Wind Speed
23,Kaitangata,0,NZ,1592242032,78,-46.28,169.85,45.00,4.00
95,Leningradskiy,0,RU,1592242045,86,69.38,178.42,42.58,2.06
114,San Julián,0,MX,1592242018,37,21.02,-102.17,72.99,7.40
129,Goderich,0,CA,1592242277,49,43.75,-81.72,73.99,1.99
137,Lavumisa,0,SZ,1592242279,55,-27.31,31.89,66.20,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
#Store good_weath dataframe into hotel_df
hotel_df = good_weath
#Add hotel name column to the datafame
hotel_df["Hotel Name"] = ""
#Reset the index of the dataframe
hotel_df = hotel_df.reset_index()
#Del index column 
del hotel_df['index']
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max temp,Wind Speed,Hotel Name
0,Kaitangata,0,NZ,1592242032,78,-46.28,169.85,45.00,4.00,
1,Leningradskiy,0,RU,1592242045,86,69.38,178.42,42.58,2.06,
2,San Julián,0,MX,1592242018,37,21.02,-102.17,72.99,7.40,
3,Goderich,0,CA,1592242277,49,43.75,-81.72,73.99,1.99,
4,Lavumisa,0,SZ,1592242279,55,-27.31,31.89,66.20,2.24,


In [14]:
#base_url for search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#type of search is lodging (hotel), radius is 5000 miles
params = {"radius": "5000",
    "type": "lodging",
    "key": g_key,}
#loop through the dataframe by each row
for index, row in hotel_df.iterrows():
    # get lat/long
    lat = row['Lat']
    long = row['Long']

    # add lat/long to params dict
    params['location'] = f"{lat},{long}"
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    result = response['results']
    # extract results
    try:
        hotel = result[0]['name']
        hotel_df.loc[index, 'Hotel Name'] = hotel
    except (TypeError,IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "NaN"
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max temp,Wind Speed,Hotel Name
0,Kaitangata,0,NZ,1592242032,78,-46.28,169.85,45.00,4.00,Kaitangata Motor Camp
1,Leningradskiy,0,RU,1592242045,86,69.38,178.42,42.58,2.06,NaN
2,San Julián,0,MX,1592242018,37,21.02,-102.17,72.99,7.40,La Nueva España
3,Goderich,0,CA,1592242277,49,43.75,-81.72,73.99,1.99,Harmony Inn
4,Lavumisa,0,SZ,1592242279,55,-27.31,31.89,66.20,2.24,ROADSIDE B&B
5,Leirvik,0,NO,1592242282,59,59.78,5.50,72.00,1.99,Stord Hotel
6,Broken Hill,0,AU,1592242262,57,-31.95,141.43,51.80,9.17,Daydream Motel & Cottages
7,Palana,0,RU,1592242284,73,59.12,159.97,49.77,2.53,NaN
8,Kiryat Gat,0,IL,1592242285,30,31.61,34.76,77.00,7.00,רגע בנחת
9,Lobatse,0,BW,1592242288,27,-25.22,25.67,53.60,2.24,Cumberland Hotel


In [15]:
#Get rid of locations with no value
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="NaN"]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max temp,Wind Speed,Hotel Name
0,Kaitangata,0,NZ,1592242032,78,-46.28,169.85,45.00,4.00,Kaitangata Motor Camp
2,San Julián,0,MX,1592242018,37,21.02,-102.17,72.99,7.40,La Nueva España
3,Goderich,0,CA,1592242277,49,43.75,-81.72,73.99,1.99,Harmony Inn
4,Lavumisa,0,SZ,1592242279,55,-27.31,31.89,66.20,2.24,ROADSIDE B&B
5,Leirvik,0,NO,1592242282,59,59.78,5.50,72.00,1.99,Stord Hotel
6,Broken Hill,0,AU,1592242262,57,-31.95,141.43,51.80,9.17,Daydream Motel & Cottages
8,Kiryat Gat,0,IL,1592242285,30,31.61,34.76,77.00,7.00,רגע בנחת
9,Lobatse,0,BW,1592242288,27,-25.22,25.67,53.60,2.24,Cumberland Hotel
10,eMkhomazi,0,ZA,1592242289,67,-30.21,30.80,63.00,4.00,Umkomaas Guest House
12,Tommot,0,RU,1592242304,88,58.96,126.29,45.59,1.12,Gostinitsa Tommot


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]].astype(float)

In [17]:
#Create points for hotels found in good weather cities 
hotel_layer = gmaps.symbol_layer(
    locations,info_box_content=hotel_info, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2
    )
#Create map 
fig = gmaps.figure()
# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))